In [ ]:
# imports

import sys
import os
import json
import numpy as np
import pandas as pd
import importlib
sys.path.append('../')
import plotting.plottools
importlib.reload(plotting.plottools)
from plotting.plottools import plot_histogram
import normalizing
importlib.reload(normalizing)
from normalizing import normalize_by_omsjson
import rebinning
importlib.reload(rebinning)
from rebinning import rebin_keep_zero
import averaging
importlib.reload(averaging)
from averaging import average_outer_ring
from averaging import average_rings

In [ ]:
# get some example histograms

fname = '../data/examples/ZeroBias-Run2023C-PromptReco-v1-DQMIO-PixelPhase1-Tracks-PXForward-clusterposition_xy_ontrack_PXDisk_+1.parquet'
df = pd.read_parquet(fname)
xbins = df['Xbins'][0]
ybins = df['Ybins'][0]
nhists = 5
hists = np.array([df['histo'][i].reshape(xbins,ybins) for i in range(nhists)])
runs = np.array(df['fromrun'])
lumis = np.array(df['fromlumi'])
entries = np.array(df['entries'])
print('Shape of hists array: {}'.format(hists.shape))
print('Runs: {}'.format(runs))
print('Lumis: {}'.format(lumis))
print('Entries: {}'.format(entries))

In [ ]:
# plot raw data

for i, hist in enumerate(hists): 
    fig, ax = plot_histogram(hist, figsize=(5,5), xaxtitle='x (bin number)', xaxtitlesize=12, yaxtitle='y (bin number)', yaxtitlesize=12, caxtitle='Occupancy', caxtitlesize=15, caxtitleoffset=20)
    ax.text(0.02, 1.02, 'Run: {}, lumi: {}'.format(runs[i], lumis[i]), transform=ax.transAxes, fontsize=12)
    fig.tight_layout()
    #fig.savefig('raw_{}.png'.format(i), dpi=300)

In [ ]:
# normalize by pileup and plot

omsfile = 'omsdata/omsdata_Run2023C-v1.json'
with open(omsfile, 'r') as f:
    omsjson = json.load(f)
print(list(omsjson.keys()))

runmask = np.isin(omsjson['run_number'], runs)
lumimask = np.isin(omsjson['lumisection_number'], lumis)
mask = (runmask & lumimask)
print('Indices: {}'.format(mask.nonzero()[0]))
for key in omsjson.keys():
    values = np.array(omsjson[key])[mask]
    print('{}: {}'.format(key, values))
    
hists = normalize_by_omsjson(hists, runs, lumis, omsjson, 'pileup')

for i, hist in enumerate(hists):
    fig, ax = plot_histogram(hist, figsize=(5,5), xaxtitle='x (bin number)', xaxtitlesize=12, yaxtitle='y (bin number)', yaxtitlesize=12, caxtitle='Occupancy', caxtitlesize=15, caxtitleoffset=20)
    ax.text(0.02, 1.02, 'Run: {}, lumi: {}, normalized by pileup'.format(runs[i], lumis[i]), transform=ax.transAxes, fontsize=12)
    fig.tight_layout()
    #fig.savefig('normalized_{}.png'.format(i), dpi=300)

In [ ]:
# make average of outer rings and plot

hists = average_rings(hists, radii={0:2, 50:5})
for i, hist in enumerate(hists):
    fig, ax = plot_histogram(hist, figsize=(5,5), xaxtitle='x (bin number)', xaxtitlesize=12, yaxtitle='y (bin number)', yaxtitlesize=12, caxtitle='Occupancy', caxtitlesize=15, caxtitleoffset=20)
    ax.text(0.02, 1.02, 'Run: {}, lumi: {}, averaged'.format(runs[i], lumis[i]), transform=ax.transAxes, fontsize=12)
    fig.tight_layout()
    #fig.savefig('averaged_{}.png'.format(i), dpi=300)

In [ ]:
# do rebinning on one example histogram

hist = hists[1]
rebinned_hist = rebin_keep_zero(hist, (40,40))
plot_histogram(rebinned_hist, figsize=(5,5), xaxtitle='x (bin number)', xaxtitlesize=12, yaxtitle='y (bin number)', yaxtitlesize=12, caxtitle='Occupancy', caxtitlesize=15, caxtitleoffset=20)

In [ ]:
# do rebinning for all histograms at once

rebinned_hists = rebin_keep_zero(hists, (40, 40))
for i, hist in enumerate(rebinned_hists):
    fig, ax = plot_histogram(hist, figsize=(5,5), xaxtitle='x (bin number)', xaxtitlesize=12, yaxtitle='y (bin number)', yaxtitlesize=12, caxtitle='Occupancy', caxtitlesize=15, caxtitleoffset=20)
    ax.text(0.02, 1.02, 'Run: {}, lumi: {}, rebinned'.format(runs[i], lumis[i]), transform=ax.transAxes, fontsize=12)
    fig.tight_layout()
    #fig.savefig('rebinned_{}.png'.format(i), dpi=300)